#1. 데이터 확인

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os

path = '/content/drive/MyDrive/17. 이미지 분석 2/cats_and_dogs/data/train/cats'

# os.listdir(path)
len(os.listdir(path))

train cats 이미지 전체 개수 :  1000
train dogs 이미지 전체 개수 :  1000
test cats 이미지 전체 개수 :  500
test dogs 이미지 전체 개수 :  500
validation cats 이미지 전체 개수 :  500
validation dogs 이미지 전체 개수 :  500


##  quiz
> 다음과 같은 결과가 나올 수 있도록 코딩하시오
- test   cats  이미지 전체 개수 :  500
- test   dogs  이미지 전체 개수 :  500
- train   cats  이미지 전체 개수 :  1000
- train   dogs  이미지 전체 개수 :  1000
- validation   cats  이미지 전체 개수 :  500
- validation   dogs  이미지 전체 개수 :  500

In [ ]:
import os

path = '/content/drive/MyDrive/17. 이미지 분석 2/cats_and_dogs/data/'

data_list=['train', 'test', 'validation']
animal_list =['cats', 'dogs']

for dir in data_list:
  for animal in animal_list:
    # print(path + dir + '/' + animal)
    print(dir, animal, '이미지 전체 개수 : ', len(os.listdir(path + dir + '/' + animal)))

#2. 모델 만들기

##2.1 모델 구성

In [9]:
from keras import models
from keras import layers

model = models.Sequential()

# hidden layer에 합성곱과 맥스 풀링 설정
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

# 1차원으로 변환
model.add(layers.Flatten())

# 결과얻기
# 고양이와 개 중 하나를 고르는 것으로 sigmoid 설정
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

##2.2 컴파일 구성

In [18]:
from keras import optimizers

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
from tensorflow.keras import optimizers, losses, metrics

model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss=losses.BinaryCrossentropy(),
              metrics=[metrics.BinaryAccuracy(),
                       metrics.FalseNegatives()])

#3. 데이터 전처리
> 데이터를 모델 전달 시 부동 소수 타입의 텐서로 적절히 변환해야 한다. 현재 데이터는 이미지 데이터로 정수형으로 되어 있으며 다음 단계로 부동 소수 타입의 텐서로 변경 가능하다.
- 사진 읽기
- 이미지를 RGB 빅셀로 디코딩
- 디코딩한 내용을 부동 소수 타입으로 변환

> 위와 같은 기능을 하는 것이 ImageDataGenerator 클래스 이다.

##3.1 부동 소수 타입의 텐서 변환

In [20]:
from keras.preprocessing.image import ImageDataGenerator

# X_train.astype('float32')/255 이와 같이 실수화 처리함.
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

##3.2 generator 만들기

In [21]:
train_dir = path + 'train'
train_dir

'/content/drive/MyDrive/17. 이미지 분석 2/cats_and_dogs/data/train'

In [23]:
train_generator = train_datagen.flow_from_directory(
    # 이미지 경로
    train_dir, 
    # 사용할 이미지 크기
    target_size=(150, 150),
    # 배치 크기
    batch_size=20,
    # 손실 데이터 연관
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


> class_mode는 분류 방식에 대해 지정한다.
- categorical : 2D ont-hot 
- binary : 1D 이진 데이터
- sparse : 1D 정수 데이터
- None : 레이블이 반환되지 않음

In [25]:
validation_dir = path + 'validation'

validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


#4. 모델 만들기

In [ ]:
history = model.fit_generator(
    # 훈련데이터
    train_generator,
    # 훈련데이터가 2000개이고 batch_size가 20이기 때문에 100으로 지정
    steps_per_epoch = 100, 
    # 반복 횟수
    epochs=30,
    # 검증 데이터
    validation_data=validation_generator,
    # 검증데이터가 1000개이고 batch_size가 20이기 때문에 50으로 지정
    validation_steps=50
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 1372s 14s/step - loss: 0.7711 - acc: 0.5160 - val_loss: 0.6777 - val_acc: 0.5990
Epoch 2/30
100/100 [==============================] - 112s 1s/step - loss: 0.6532 - acc: 0.6305 - val_loss: 0.6436 - val_acc: 0.6410
Epoch 3/30
 13/100 [==>...........................] - ETA: 1:24 - loss: 0.6545 - acc: 0.6000

#5. 백업

##5.1 모델 저장

In [ ]:
model_path = '/content/drive/MyDrive/17. 이미지 분석 2/model/cats_and_dogs_small_1.h5'
model.save(model_path)

##5.2 히스토리 저장

In [ ]:
import numpy as np
history_path = '/content/drive/MyDrive/17. 이미지 분석 2/model/my_history.npy'
np.save(history_path, history.history)